# Chargement des données

In [1]:
# mise en place des chemins des fichiers contenant les datasets
dossier_data = "data/brutes/"
nom_fichier_application_train = "application_train.csv"
nom_fichier_application_test = "application_test.csv"
nom_fichier_bureau = "bureau.csv"
nom_fichier_bureau_balance = "bureau_balance.csv"
nom_fichier_POS_CASH_balance = "POS_CASH_balance.csv"
nom_fichier_credit_card_balance = "credit_card_balance.csv"
nom_fichier_previous_application = "previous_application.csv"
nom_fichier_installments_payments = "installments_payments.csv"

In [2]:
import pandas as pd

# chargement des datasets (BDD complète)
application_train = pd.read_csv(dossier_data + nom_fichier_application_train)
application_test = pd.read_csv(dossier_data + nom_fichier_application_test)
bureau = pd.read_csv(dossier_data + nom_fichier_bureau)
#bureau_balance = pd.read_csv(dossier_data + nom_fichier_bureau_balance)
#POS_CASH_balance = pd.read_csv(dossier_data + nom_fichier_POS_CASH_balance)
#credit_card_balance = pd.read_csv(dossier_data + nom_fichier_credit_card_balance)
previous_application = pd.read_csv(dossier_data + nom_fichier_previous_application)
#installments_payments = pd.read_csv(dossier_data + nom_fichier_installments_payments)

In [3]:
pd.set_option("display.max_columns", None)
previous_application

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.000000,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670209,2300464,352015,Consumer loans,14704.290,267295.5,311400.0,0.0,267295.5,WEDNESDAY,12,Y,1,0.000000,NaN,NaN,XAP,Approved,-544,Cash through the bank,XAP,NaN,Refreshed,Furniture,POS,XNA,Stone,43,Furniture,30.0,low_normal,POS industry with interest,365243.0,-508.0,362.0,-358.0,-351.0,0.0
1670210,2357031,334635,Consumer loans,6622.020,87750.0,64291.5,29250.0,87750.0,TUESDAY,15,Y,1,0.340554,NaN,NaN,XAP,Approved,-1694,Cash through the bank,XAP,Unaccompanied,New,Furniture,POS,XNA,Stone,43,Furniture,12.0,middle,POS industry with interest,365243.0,-1604.0,-1274.0,-1304.0,-1297.0,0.0
1670211,2659632,249544,Consumer loans,11520.855,105237.0,102523.5,10525.5,105237.0,MONDAY,12,Y,1,0.101401,NaN,NaN,XAP,Approved,-1488,Cash through the bank,XAP,"Spouse, partner",Repeater,Consumer Electronics,POS,XNA,Country-wide,1370,Consumer electronics,10.0,low_normal,POS household with interest,365243.0,-1457.0,-1187.0,-1187.0,-1181.0,0.0
1670212,2785582,400317,Cash loans,18821.520,180000.0,191880.0,NaN,180000.0,WEDNESDAY,9,Y,1,NaN,NaN,NaN,XNA,Approved,-1185,Cash through the bank,XAP,Family,Repeater,XNA,Cash,x-sell,AP+ (Cash loan),-1,XNA,12.0,low_normal,Cash X-Sell: low,365243.0,-1155.0,-825.0,-825.0,-817.0,1.0


Au vu des données, je ne vois pas comment utiliser les tables listant les mensualités des crédits précédents pour lesquelles peu d'infos sont disponibles (cf. *POS_CASH_balance*, *bureau_balance* et *installments_payments*). 

De ce fait, les tables qui semblent exploitables sont les suivantes :
- *application_{train|test}*
- *bureau*
- *previous_application*

Les autres ne seront plus chargées dans le notebook.

Avec les données des précédents crédits répertoriés par Credit Bureau, on pourrait voir :
- combien de crédits le client a eu => réel intérêt ? Seulement si on réalise un taux de nb de crédits avec retard de paiement sur nb de crédit, à voir si on a assez de données sur la table principale
- si le client a eu des retards de paiement et si oui de quel ordre
- combien de crédits le client a actuellement (donnée la plus importante parmi les 3)

Alors qu'avec les données des demandes précédentes de crédits faits à Home Credit, on pourrait voir :
- combien de crédits le client a eu
- combien de crédits le client a actuellement (donnée la plus importante parmi les 3)
- combien de demandes de crédit ont déjà été refusées au client par Home Credit

Ces données ne seront utilisées seulement dans le cas où j'ai un peu plus de temps. En effet, afin de réaliser la modélisation au plus vite, j'utiliserai dans un premier temps les données des demandes de crédit chez Home Credit.

# Préparation des données

## Précédents crédits répertoriés par Credit Bureau (partie à supprimer si pas eu le temps de faire du features engineering)

In [41]:
bureau.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716428 entries, 0 to 1716427
Data columns (total 17 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_CURR              int64  
 1   SK_ID_BUREAU            int64  
 2   CREDIT_ACTIVE           object 
 3   CREDIT_CURRENCY         object 
 4   DAYS_CREDIT             int64  
 5   CREDIT_DAY_OVERDUE      int64  
 6   DAYS_CREDIT_ENDDATE     float64
 7   DAYS_ENDDATE_FACT       float64
 8   AMT_CREDIT_MAX_OVERDUE  float64
 9   CNT_CREDIT_PROLONG      int64  
 10  AMT_CREDIT_SUM          float64
 11  AMT_CREDIT_SUM_DEBT     float64
 12  AMT_CREDIT_SUM_LIMIT    float64
 13  AMT_CREDIT_SUM_OVERDUE  float64
 14  CREDIT_TYPE             object 
 15  DAYS_CREDIT_UPDATE      int64  
 16  AMT_ANNUITY             float64
dtypes: float64(8), int64(6), object(3)
memory usage: 222.6+ MB


On regarde quelles sont les modalités correspondantes à la variable relative au statut du crédit pour savoir si on peut dire si un client a fait défaut de paiement. Etant donné que l'on ne sait pas pourquoi un crédit a été vendu (parce qu'il était mauvais ? Ou pour toute autre raison ?), on ne peut pas savoir dans ce dataset si un client a fait défaut de paiement.

In [33]:
bureau.CREDIT_ACTIVE.value_counts()

Closed      1079273
Active       630607
Sold           6527
Bad debt         21
Name: CREDIT_ACTIVE, dtype: int64

Sur ce jeu de données, nous allons conserver les variables suivantes :
- SK_ID_CURR
- SK_ID_BUREAU
- CREDIT_ACTIVE
- CREDIT_DAY_OVERDUE
- AMT_CREDIT_SUM
- AMT_CREDIT_SUM_OVERDUE

Elles permettent de créer les données citées dans la partie précédente.

In [44]:
variables_a_conserver = [
    "SK_ID_CURR",
    "SK_ID_BUREAU",
    "CREDIT_ACTIVE",
    "CREDIT_DAY_OVERDUE",
    "AMT_CREDIT_SUM",
    "AMT_CREDIT_SUM_OVERDUE"
]
precedents_credits_CB = bureau[variables_a_conserver].copy()

In [45]:
precedents_credits_CB

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_DAY_OVERDUE,AMT_CREDIT_SUM,AMT_CREDIT_SUM_OVERDUE
0,215354,5714462,Closed,0,91323.00,0.0
1,215354,5714463,Active,0,225000.00,0.0
2,215354,5714464,Active,0,464323.50,0.0
3,215354,5714465,Active,0,90000.00,0.0
4,215354,5714466,Active,0,2700000.00,0.0
...,...,...,...,...,...,...
1716423,259355,5057750,Active,0,11250.00,0.0
1716424,100044,5057754,Closed,0,38130.84,0.0
1716425,100044,5057762,Closed,0,15570.00,0.0
1716426,246829,5057770,Closed,0,36000.00,0.0


## Demandes de crédit en cours chez Home Credit

In [7]:
application_train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.0220,0.0198,0.0,0.0000,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.0000,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.0790,0.0554,0.0,0.0000,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.0100,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [19]:
credits.TARGET.value_counts()

0    282686
1     24825
Name: TARGET, dtype: int64

In [10]:
credits.TARGET.value_counts(normalize=True)

0    0.919271
1    0.080729
Name: TARGET, dtype: float64

Il faudra porter une attention particulière lors du split du dataset et de la suppression éventuelle de données car le dataset est déséquilibré.

### Sélection des variables

Les données conservées dans la table principale (train) sont les suivantes :
- *SK_ID_CURR* : ID du crédit
- *TARGET* : variable à expliquer, vaut 1 si le client a eu au moins un retard de paiement, 0 sinon 
- *NAME_CONTRACT_TYPE* : identifie si le crédit est en "cash" ou "revolving"
- *CODE_GENDER* : genre du client
- *FLAG_OWN_CAR* : flag si le client possède une voiture
- *FLAG_OWN_REALTY* : flag si le client possède une maison ou un appartement
- *CNT_CHILDREN* : nombre d'enfants du client
- *AMT_INCOME_TOTAL* : revenu du client
- *AMT_CREDIT* : montant du crédit
- *AMT_ANNUITY* : montant annuel à rembourser par le client
- *AMT_GOODS_PRICE* : si c'est un crédit à la consommation, montant des biens pour lequel le client fait crédit
- *NAME_INCOME_TYPE* : source de revenu du client (job, retraite, etc.)
- *NAME_FAMILY_STATUS* : statut familial du client (divorcé, marié, célibataire, etc.)
- *NAME_HOUSING_TYPE* : statut relatif au logement du client (propriétaire, locataire, hébergé, etc.) 
- *DAYS_BIRTH* : âge du client (en jours) au moment de la demande
- *DAYS_EMPLOYED* : durée de l'emploi du client en jours
- *OWN_CAR_AGE* : âge de la voiture du client (en années ?)
- *OCCUPATION_TYPE* : type d'activité du client
- *CNT_FAM_MEMBERS* : nombre de membres de la famille du client
- *ORGANIZATION_TYPE* : type de l'organisation dans laquelle le client travaille
- *OBS_30_CNT_SOCIAL_CIRCLE* : nombre de personnes de l'entourage du client avec un retard de paiement de 30 jours
- *DEF_30_CNT_SOCIAL_CIRCLE* : nombre de personnes de l'entourage du client en défaut de paiement à 30 jours
- *OBS_60_CNT_SOCIAL_CIRCLE* : nombre de personnes de l'entourage du client avec un retard de paiement de 60 jours
- *DEF_60_CNT_SOCIAL_CIRCLE* : nombre de personnes de l'entourage du client en défaut de paiement à 60 jours

In [8]:
# Conservation des variables d'entrée pour les modèles
variables_a_conserver = [
    "SK_ID_CURR",
    "TARGET",
    "NAME_CONTRACT_TYPE",
    "CODE_GENDER",
    "FLAG_OWN_CAR",
    "FLAG_OWN_REALTY",
    "CNT_CHILDREN",
    "AMT_INCOME_TOTAL",
    "AMT_CREDIT",
    "AMT_ANNUITY",
    "AMT_GOODS_PRICE",
    "NAME_INCOME_TYPE",
    "NAME_FAMILY_STATUS",
    "NAME_HOUSING_TYPE",
    "DAYS_BIRTH",
    "DAYS_EMPLOYED",
    "OWN_CAR_AGE",
    "OCCUPATION_TYPE",
    "CNT_FAM_MEMBERS",
    "ORGANIZATION_TYPE",
    "OBS_30_CNT_SOCIAL_CIRCLE",
    "DEF_30_CNT_SOCIAL_CIRCLE",
    "OBS_60_CNT_SOCIAL_CIRCLE",
    "DEF_60_CNT_SOCIAL_CIRCLE"
]

credits = application_train[variables_a_conserver].copy()

In [11]:
credits

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,OWN_CAR_AGE,OCCUPATION_TYPE,CNT_FAM_MEMBERS,ORGANIZATION_TYPE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Working,Single / not married,House / apartment,-9461,-637,NaN,Laborers,1.0,Business Entity Type 3,2.0,2.0,2.0,2.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,State servant,Married,House / apartment,-16765,-1188,NaN,Core staff,2.0,School,1.0,0.0,1.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Working,Single / not married,House / apartment,-19046,-225,26.0,Laborers,1.0,Government,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Working,Civil marriage,House / apartment,-19005,-3039,NaN,Laborers,2.0,Business Entity Type 3,2.0,0.0,2.0,0.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Working,Single / not married,House / apartment,-19932,-3038,NaN,Core staff,1.0,Religion,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,225000.0,Working,Separated,With parents,-9327,-236,NaN,Sales staff,1.0,Services,0.0,0.0,0.0,0.0
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,225000.0,Pensioner,Widow,House / apartment,-20775,365243,NaN,NaN,1.0,XNA,0.0,0.0,0.0,0.0
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,585000.0,Working,Separated,House / apartment,-14966,-7921,NaN,Managers,1.0,School,6.0,0.0,6.0,0.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,319500.0,Commercial associate,Married,House / apartment,-11961,-4786,NaN,Laborers,2.0,Business Entity Type 1,0.0,0.0,0.0,0.0


### Données manquantes

In [12]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SK_ID_CURR                307511 non-null  int64  
 1   TARGET                    307511 non-null  int64  
 2   NAME_CONTRACT_TYPE        307511 non-null  object 
 3   CODE_GENDER               307511 non-null  object 
 4   FLAG_OWN_CAR              307511 non-null  object 
 5   FLAG_OWN_REALTY           307511 non-null  object 
 6   CNT_CHILDREN              307511 non-null  int64  
 7   AMT_INCOME_TOTAL          307511 non-null  float64
 8   AMT_CREDIT                307511 non-null  float64
 9   AMT_ANNUITY               307499 non-null  float64
 10  AMT_GOODS_PRICE           307233 non-null  float64
 11  NAME_INCOME_TYPE          307511 non-null  object 
 12  NAME_FAMILY_STATUS        307511 non-null  object 
 13  NAME_HOUSING_TYPE         307511 non-null  o

#### Âge de la voiture

Presque 2/3 des données sont manquantes dans cette variable. Vérifions si cela vient bien du fait qu'un client n'a pas de voiture.

In [14]:
credits["FLAG_EMPTY_CAR_AGE"] = credits["OWN_CAR_AGE"].isnull()

In [18]:
analyse_age_voiture_vide = pd.crosstab(credits["FLAG_EMPTY_CAR_AGE"], credits["FLAG_OWN_CAR"], margins=True)
analyse_age_voiture_vide

FLAG_OWN_CAR,N,Y,All
FLAG_EMPTY_CAR_AGE,,,
False,0,104582,104582
True,202924,5,202929
All,202924,104587,307511


Presque la totalité des âges de voitures vides proviennent du fait que le client n'a pas de voiture, sauf pour 5 d'entre eux. Ces 5 clients seront supprimés.

In [22]:
index_a_supprimer = credits[(credits["FLAG_EMPTY_CAR_AGE"] == True) & (credits["FLAG_OWN_CAR"] == 'Y')].index
index_a_supprimer

Int64Index([30897, 181231, 217549, 229867, 236868], dtype='int64')

In [23]:
# suppression des clients ayant une voiture mais sans leur âge
credits.drop(index_a_supprimer, inplace=True)

In [27]:
credits.TARGET.value_counts()

0    282681
1     24825
Name: TARGET, dtype: int64

Afin d'avoir une variable sans donnée manquante, une nouvelle variable sera construite à partir du flag désignant le fait de posséder une voiture et de l'âge du véhicule.

#### Type de l'activité

Presque 1/3 des données sont manquantes dans cette variable. Vérifions si cela vient bien du fait qu'un client n'a pas de travail (chômage, retraite, congés longue durée, etc.).

In [29]:
# flag sur les données manquantes du type d'activité
credits["FLAG_EMPTY_OCCUPATION_TYPE"] = credits["OCCUPATION_TYPE"].isnull()

In [30]:
credits.NAME_INCOME_TYPE.value_counts()

Working                 158772
Commercial associate     71614
Pensioner                55362
State servant            21703
Unemployed                  22
Student                     18
Businessman                 10
Maternity leave              5
Name: NAME_INCOME_TYPE, dtype: int64

In [31]:
analyse_type_activite_vide = pd.crosstab(credits["FLAG_EMPTY_OCCUPATION_TYPE"], credits["NAME_INCOME_TYPE"], margins=True)
analyse_type_activite_vide

NAME_INCOME_TYPE,Businessman,Commercial associate,Maternity leave,Pensioner,State servant,Student,Unemployed,Working,All
FLAG_EMPTY_OCCUPATION_TYPE,,,,,,,,,
False,8,59318,4,5,17916,13,0,133852,211116
True,2,12296,1,55357,3787,5,22,24920,96390
All,10,71614,5,55362,21703,18,22,158772,307506


En effet, il y a bien des données manquantes sur le type de l'activité pour quasiment l'ensemble des retraités et des chômeurs. Cependant, il reste trop de données manquantes pour d'autres clients tels que les associés, les fonctionnaires ou les employés (*working*).

In [34]:
# on regarde si des tendances se dégagent dans d'autres variables
credits[credits["OCCUPATION_TYPE"].isnull()].sample(15)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,OWN_CAR_AGE,OCCUPATION_TYPE,CNT_FAM_MEMBERS,ORGANIZATION_TYPE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,FLAG_EMPTY_CAR_AGE,FLAG_EMPTY_OCCUPATION_TYPE
87139,201136,0,Cash loans,F,N,Y,0,67500.0,152820.0,9895.5,135000.0,Pensioner,Married,House / apartment,-21380,365243,NaN,NaN,2.0,XNA,2.0,0.0,2.0,0.0,True,True
102612,219117,0,Cash loans,M,Y,N,0,80302.5,1147500.0,29007.0,1147500.0,Commercial associate,Married,House / apartment,-20597,-153,14.0,NaN,2.0,Trade: type 3,1.0,0.0,1.0,0.0,False,True
143273,266134,0,Revolving loans,F,Y,Y,1,135000.0,180000.0,9000.0,180000.0,State servant,Married,House / apartment,-9980,-962,7.0,NaN,3.0,Security Ministries,2.0,1.0,2.0,0.0,False,True
164095,290228,0,Cash loans,F,N,Y,1,202500.0,1024740.0,52452.0,900000.0,Commercial associate,Separated,House / apartment,-15746,-691,NaN,NaN,2.0,Hotel,2.0,1.0,2.0,1.0,True,True
13331,115544,0,Cash loans,M,N,N,0,126000.0,239850.0,23719.5,225000.0,Pensioner,Separated,House / apartment,-18190,365243,NaN,NaN,1.0,XNA,3.0,0.0,3.0,0.0,True,True
207828,340890,0,Cash loans,M,N,Y,0,63000.0,78192.0,8338.5,67500.0,Pensioner,Married,House / apartment,-23610,365243,NaN,NaN,2.0,XNA,0.0,0.0,0.0,0.0,True,True
146291,269616,0,Cash loans,F,N,Y,1,180000.0,991678.5,35748.0,837000.0,Working,Married,House / apartment,-16354,-7380,NaN,NaN,3.0,Other,0.0,0.0,0.0,0.0,True,True
92214,207080,0,Revolving loans,M,N,N,0,135000.0,180000.0,9000.0,180000.0,Working,Married,House / apartment,-8448,-824,NaN,NaN,2.0,Business Entity Type 3,3.0,0.0,3.0,0.0,True,True
106948,224078,0,Cash loans,F,N,N,0,67500.0,229500.0,10048.5,229500.0,Pensioner,Widow,House / apartment,-20267,365243,NaN,NaN,1.0,XNA,0.0,0.0,0.0,0.0,True,True
196654,328020,0,Cash loans,F,Y,N,0,427500.0,1546020.0,44437.5,1350000.0,Commercial associate,Single / not married,House / apartment,-15868,-2229,4.0,NaN,1.0,Business Entity Type 3,1.0,0.0,1.0,0.0,False,True


RAS sur le dataset réduit.

In [35]:
# on fait la même chose sur le dataset initial
application_train[application_train["OCCUPATION_TYPE"].isnull()].sample(15)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
120575,239804,0,Cash loans,F,N,Y,3,112500.0,594121.5,26167.5,472500.0,Family,Pensioner,Secondary / secondary special,Widow,House / apartment,0.010276,-20768,365243,-12151.0,-1099,NaN,1,0,0,1,0,0,NaN,4.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,XNA,0.277007,0.576165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
147313,270800,0,Cash loans,F,N,Y,0,247500.0,572148.0,41764.5,517500.0,Unaccompanied,Pensioner,Higher education,Married,House / apartment,0.016612,-18775,365243,-6344.0,-2322,NaN,1,0,0,1,0,0,NaN,2.0,2,2,WEDNESDAY,11,0,0,0,0,0,0,XNA,0.841436,0.715011,0.342529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,1.0,5.0,1.0,-1934.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0
199629,331430,0,Cash loans,F,N,Y,0,85500.0,110331.0,10876.5,103500.0,Unaccompanied,Pensioner,Secondary / secondary special,Widow,House / apartment,0.031329,-24619,365243,-2222.0,-4425,NaN,1,0,0,1,0,0,NaN,1.0,2,2,FRIDAY,10,0,0,0,0,0,0,XNA,0.848245,0.191526,0.819318,0.0495,NaN,0.9752,NaN,NaN,NaN,0.1034,0.1250,NaN,NaN,NaN,0.0385,NaN,0.0126,0.0504,NaN,0.9752,NaN,NaN,NaN,0.1034,0.1250,NaN,NaN,NaN,0.0402,NaN,0.0133,0.0500,NaN,0.9752,NaN,NaN,NaN,0.1034,0.1250,NaN,NaN,NaN,0.0392,NaN,0.0129,NaN,block of flats,0.0331,"Stone, brick",No,0.0,0.0,0.0,0.0,-1124.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
83453,196794,0,Cash loans,F,N,Y,0,90000.0,269550.0,19300.5,225000.0,Unaccompanied,Commercial associate,Secondary / 

RAS non plus sur le dataset initial.

Au vu du nombre de données manquantes non expliquées (environ 13% des clients), la variable sera supprimée. Je me servirai de la variable *NAME_INCOME_TYPE*.

In [38]:
# suppression de la variable
credits.drop(columns=["OCCUPATION_TYPE", "FLAG_EMPTY_OCCUPATION_TYPE"], inplace=True)

In [39]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307506 entries, 0 to 307510
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SK_ID_CURR                307506 non-null  int64  
 1   TARGET                    307506 non-null  int64  
 2   NAME_CONTRACT_TYPE        307506 non-null  object 
 3   CODE_GENDER               307506 non-null  object 
 4   FLAG_OWN_CAR              307506 non-null  object 
 5   FLAG_OWN_REALTY           307506 non-null  object 
 6   CNT_CHILDREN              307506 non-null  int64  
 7   AMT_INCOME_TOTAL          307506 non-null  float64
 8   AMT_CREDIT                307506 non-null  float64
 9   AMT_ANNUITY               307494 non-null  float64
 10  AMT_GOODS_PRICE           307228 non-null  float64
 11  NAME_INCOME_TYPE          307506 non-null  object 
 12  NAME_FAMILY_STATUS        307506 non-null  object 
 13  NAME_HOUSING_TYPE         307506 non-null  o